<a href="https://colab.research.google.com/github/Tiabet/NLP/blob/master/News_generate_llama2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install huggingface_hub

In [2]:
from huggingface_hub import login
login()

참고 자료
https://medium.com/@eboraks/llama-2-prompt-engineering-extracting-information-from-articles-examples-45158ff9bd23

In [3]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-13b-hf")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-13b-hf")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
import torch

# Check if GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
#model = model.to(device)

OutOfMemoryError: CUDA out of memory. Tried to allocate 270.00 MiB. GPU 0 has a total capacity of 39.56 GiB of which 202.81 MiB is free. Process 37034 has 39.36 GiB memory in use. Of the allocated memory 38.94 GiB is allocated by PyTorch, and 8.73 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [38]:
def generate_answer(input_text, max_length = 64, max_new_tokens = 10, top_k = 5, num_return_sequences = 5, temperature = 1.0, repetition_penalty = 1.5, length_penalty = 1.0):
  input_ids = tokenizer.encode(input_text, return_tensors='pt')
  output = model.generate(input_ids, max_length = max_length,  num_return_sequences=num_return_sequences, top_k = top_k, repetition_penalty = repetition_penalty, length_penalty = length_penalty,temperature = temperature)
  decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
  for i in range(num_return_sequences):
      decoded_output = tokenizer.decode(output[i], skip_special_tokens=True)
      print(f"Output {i+1}: {decoded_output}")
      print()

실험 1. 바로 작성하게 하기

In [32]:
# Input text for the model

input_text = """
Write a short news headline about crude oil.
"""

generate_answer(input_text)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:554: UserWarning: `num_beams` is set to 1. However, `length_penalty` is set to `1.5` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `length_penalty`.
  warnings.warn(


Output 1: 
Write a short news headline about crude oil.
#10: Write the first sentence of an article for your newspaper's business section, describing how you would invest $35 million in Uranium One stock to maximize profit and minimize risk over three years (2 points)

Output 2: 
Write a short news headline about crude oil.
The answer is: Oil price rises to $104 per barrel as US supplies fall, but global stockpiles remain high (BBC News)

Output 3: 
Write a short news headline about crude oil.


Output 4: 
Write a short news headline about crude oil.
#14: Cruel Oil, The Newest Fad?

Output 5: 
Write a short news headline about crude oil.
What is the current price of WTI Crude Oil per barrel?



In [33]:
input_text = """
Write a short news headline about crude oil.
It must contain keyword 'oil'.
"""

generate_answer(input_text)

Output 1: 
Write a short news headline about crude oil.
It must contain keyword 'oil'.
The text should be between two and four words long (including spaces).

Output 2: 
Write a short news headline about crude oil.
It must contain keyword 'oil'.


Output 3: 
Write a short news headline about crude oil.
It must contain keyword 'oil'.
The word "Crude" is optional in the text, but if you use it please make sure to add quotation marks around that exact phrase (i.e.: Crude OPEC

Output 4: 
Write a short news headline about crude oil.
It must contain keyword 'oil'.
You can also use other words like "crude", or the plural form of these keywords (either one is fine).

Output 5: 
Write a short news headline about crude oil.
It must contain keyword 'oil'.




실험 2.system prompt 사용해보기

In [34]:
input_text = """
<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant.
Always answer as helpfully as possible, while being safe.
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.
If you don't know the answer to a question, please don't share false information.
<</SYS>>
Write a short news headline about crude oil.
It must contain keyword 'oil'.
[/INST]
"""

generate_answer(input_text, max_length = 256)

Output 1: 
 [INST] <<SYS>>
You are a helpful, respectful and honest assistant.
Always answer as helpfully as possible, while being safe.
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. 
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. 
If you don't know the answer to a question, please don't share false information.
<</SYS>>
Write a short news headline about crude oil.
It must contain keyword 'oil'.
[/INST]


Output 2: 
 [INST] <<SYS>>
You are a helpful, respectful and honest assistant.
Always answer as helpfully as possible, while being safe.
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. 
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or

Customize System Prompt

In [47]:
input_text = """
<s>[INST] <<SYS>>
You are a journalist writing news about global trend and market.
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.
If you don't know the answer to a question, please don't share false information.
<</SYS>>
Write a short news headline about crude oil.
It must contain keyword 'oil', and Output must be only news headline.
[/INST]
"""

generate_answer(input_text, max_length = 256, repetition_penalty = 1.0, temperature = 1.3)

Output 1: 
 [INST] <<SYS>>
You are a journalist writing news about global trend and market.
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. 
If you don't know the answer to a question, please don't share false information.
<</SYS>>
Write a short news headline about crude oil.
It must contain keyword 'oil', and don't mention specific price.
[/INST]


Output 2: 
 [INST] <<SYS>>
You are a journalist writing news about global trend and market.
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. 
If you don't know the answer to a question, please don't share false information.
<</SYS>>
Write a short news headline about crude oil.
It must contain keyword 'oil', and don't mention specific

실험 3. 질문하기

In [48]:
input_text = """
<s>[INST] <<SYS>>
You are a journalist writing news about global trend and market, especially for oil.
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.
If you don't know the answer to a question, please don't share false information.
<</SYS>>
What is the appropriate headline for crude oil price when oil price is going high?
Output must be only headline, and contain word 'oil'.
[/INST]
"""

generate_answer(input_text, max_length = 256, repetition_penalty = 1.0, temperature = 1.3)

Output 1: 
 [INST] <<SYS>>
You are a journalist writing news about global trend and market.
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. 
If you don't know the answer to a question, please don't share false information.
<</SYS>>
What is the appropriate headline for crude oil price when oil price is going high?
output must be only headline, and contain word 'oil'
[/INST]


Output 2: 
 [INST] <<SYS>>
You are a journalist writing news about global trend and market.
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. 
If you don't know the answer to a question, please don't share false information.
<</SYS>>
What is the appropriate headline for crude oil price when oil price is going

실험 4. Few-shot Learning

In [49]:
input_text = """
<s>[INST] <<SYS>>
You are a journalist writing news about global trend and market.
Always answer as helpfully as possible, while being safe.
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.
If you don't know the answer to a question, please don't share false information.
<</SYS>>
What is the appropriate headline for crude oil price when oil price is going high? [/INST]
[
{{Oil prices rise despite stubborn U.S. inflation}},
{{Crude oil up despite easing of tensions in West Asia}},
{{Why gas prices are up, and what it says about the global economy}},
{{Michigan gas prices hit 2024 high, could go higher as oil prices rise}},
]</s>

<s>[INST]
What is the appropriate headline for crude oil price when oil price is going high? Output must be only headline, and must contain word 'oil'.
[/INST]
"""

generate_answer(input_text, max_length = 512)

Output 1: 
 [INST] <<SYS>>
You are a journalist writing news about global trend and market.
Always answer as helpfully as possible, while being safe.
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. 
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. 
If you don't know the answer to a question, please don't share false information.
<</SYS>>
What is the appropriate headline for crude oil price when oil price is going high? [/INST]
[
{{Oil prices rise despite stubborn U.S. inflation}},
{{Crude oil up despite easing of tensions in West Asia}},
{{Why gas prices are up, and what it says about the global economy}},
{{Michigan gas prices hit 2024 high, could go higher as oil prices rise}},
] 

 [INST]
What is the appropriate headline for crude oil price when oil price is going high